# Описание проекта

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

<a id='intro'></a>

## Оглавление

[Введение](#chapter0)

[Загрузка и подготовка данных](#chapter1)

[Обучение и проверка модели](#chapter2)

[Подготовка к расчету прибыли](#chapter3)

[Расчет прибыли и рисков](#chapter4)

[Общий вывод](#chapter5)


<a id='chapter0'></a>

# Введение


Цель данной работы - спрогнозировать добычу в новых скважинах и выбрать регион, предпочтительный для разработки. Для решения этой задачи мы должны построить модель, которая сможет определить объем запасов в новых скважинах.

В качестве исходной информации нам передали данные геологоразведки - пробы нефти трех регионов.


[К оглавлению](#intro)

<a id='chapter1'></a>

# 1. Загрузка и подготовка данных

In [1]:
#загрузим необходимые библиотеки
import pandas as pd
import numpy as np
from scipy import stats as st
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

In [2]:
#загрузим данные геологоразведки
geo_data_0 = pd.read_csv('/datasets/geo_data_0.csv')
geo_data_1 = pd.read_csv('/datasets/geo_data_1.csv')
geo_data_2 = pd.read_csv('/datasets/geo_data_2.csv')
#посмотрим первые пять строк данных
geo_data_0.head()

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


In [3]:
#информация по датасету
geo_data_0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [4]:
#информация по датасету
geo_data_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [5]:
#информация по датасету
geo_data_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB



Перед нами три датасета, в каждом из которых 100000 строк. Здесь содержится информация по данным геологоразведки трех регионов.

В таблицах с данными пять столбцов:

* <i> id </i> - уникальный идентификатор скважины;

* <i> f0, f1, f2 </i> - три признака точек;

* <i> product </i> - объем запасов в скважине.



Теперь проверим данные на наличие пропущенных значений и дубликатов.


In [6]:
geo_data_0.isna().sum()

id         0
f0         0
f1         0
f2         0
product    0
dtype: int64

In [7]:
geo_data_1.isna().sum()

id         0
f0         0
f1         0
f2         0
product    0
dtype: int64

In [8]:
geo_data_2.isna().sum()

id         0
f0         0
f1         0
f2         0
product    0
dtype: int64

In [9]:
geo_data_0.duplicated().sum()

0

In [10]:
geo_data_1.duplicated().sum()

0

In [11]:
geo_data_2.duplicated().sum()

0


Полностью совпадающих строк нет, но надо также проверить, чтобы нет дублирующихся имен скважин, т.к. эти названия должны быть уникальны.


In [12]:
geo_data_0['id'].duplicated().sum()

10

In [13]:
geo_data_1['id'].duplicated().sum()

4

In [14]:
geo_data_2['id'].duplicated().sum()

4


Строки с дублирующимися идентификаторами скважин присутствуют, но их не так много, можем удалить их из общего набора данных.


In [15]:
#удаляем скважины-дубликаты и обновляем индексы
geo_data_0['id'] = geo_data_0['id'].drop_duplicates()
geo_data_0 = geo_data_0.dropna().reset_index(drop=True)
geo_data_1['id'] = geo_data_1['id'].drop_duplicates()
geo_data_1 = geo_data_1.dropna().reset_index(drop=True)
geo_data_2['id'] = geo_data_2['id'].drop_duplicates()
geo_data_2 = geo_data_2.dropna().reset_index(drop=True)

### Вывод


Итак, на данном шаге мы загрузили исходные данные и проели их предобработку. В данных нет дублирующихся строк и нет пропущенных значений, значит можем работать с ними дальше. Перед нами стоит задача регрессии, т.е. тип целевого признака количественный - надо определить объем сырья, полученный при работе скважин. Обучающий набор данных состоит из трех числовых признаков. Мы не знаем, что они означают, но по условиям известно, что сами признаки значимы.


[К оглавлению](#intro)

<a id='chapter2'></a>

# 2. Обучение и проверка модели


Сейчас нам нужно подготовить данные перед обучением модели.

Выделим признаки в каждом из трех датасетов. Целевой признак в данном случае - столбцы <i>product</i>.


In [16]:
#объявление признаков
target_0 = geo_data_0['product']
feature_0 = geo_data_0.drop(['id', 'product'], axis=1)
target_1 = geo_data_1['product']
feature_1 = geo_data_1.drop(['id', 'product'], axis=1)
target_2 = geo_data_2['product']
feature_2 = geo_data_2.drop(['id', 'product'], axis=1)


Разобъем данные на две выборки: обучающую и валидационную.


In [17]:
#выделим обучающую и валидационную выборку в первом районе
features_train_0, features_valid_0, target_train_0, target_valid_0 = (
train_test_split(feature_0, target_0, test_size=0.25, random_state=0)
)
print('Количество объектов в обучающей выборке:', len(features_train_0))
print('Количество объектов в валидационной выборке:', len(features_valid_0))

Количество объектов в обучающей выборке: 74992
Количество объектов в валидационной выборке: 24998


In [18]:
#выделим обучающую и валидационную выборку во втором районе
features_train_1, features_valid_1, target_train_1, target_valid_1 = (
train_test_split(feature_1, target_1, test_size=0.25, random_state=0)
)
print('Количество объектов в обучающей выборке:', len(features_train_1))
print('Количество объектов в валидационной выборке:', len(features_valid_1))

Количество объектов в обучающей выборке: 74997
Количество объектов в валидационной выборке: 24999


In [19]:
#выделим обучающую и валидационную выборку в третьем районе
features_train_2, features_valid_2, target_train_2, target_valid_2 = (
train_test_split(feature_2, target_2, test_size=0.25, random_state=0)
)
print('Количество объектов в обучающей выборке:', len(features_train_2))
print('Количество объектов в валидационной выборке:', len(features_valid_2))

Количество объектов в обучающей выборке: 74997
Количество объектов в валидационной выборке: 24999



Теперь, после разделения наборов данных на две выборки, нам необходимо масштабировать признаки. Если в одном разбросы больше, чем в другом, то алгоритм решит, что этот признак важнее, а это не так.


In [20]:
#численные признаки
numeric = ['f0', 'f1', 'f2']
#масштабирование
scaler = StandardScaler()
scaler.fit(features_train_0[numeric])
features_train_0[numeric] = scaler.transform(features_train_0[numeric])
features_valid_0[numeric] = scaler.transform(features_valid_0[numeric])
scaler.fit(features_train_1[numeric])
features_train_1[numeric] = scaler.transform(features_train_1[numeric])
features_valid_1[numeric] = scaler.transform(features_valid_1[numeric])
scaler.fit(features_train_2[numeric])
features_train_2[numeric] = scaler.transform(features_train_2[numeric])
features_valid_2[numeric] = scaler.transform(features_valid_2[numeric])


Обучим модели и сделаем предсказания на валидационной выборке.


In [21]:
#обучим модель по первому региону
model_0 = LinearRegression()
model_0.fit(features_train_0, target_train_0)
predict_0 = model_0.predict(features_valid_0)
rmse_0 = (mean_squared_error(target_valid_0, predict_0))**(0.5)
print('Средний запас предсказанного сырья по первому региону:', predict_0.mean())
print('Средний запас реального сырья по первому региону:', target_valid_0.mean())
print('RMSE модели:', rmse_0)

Средний запас предсказанного сырья по первому региону: 92.69630160501748
Средний запас реального сырья по первому региону: 92.7579632180877
RMSE модели: 37.7262428254728


In [22]:
#обучим модель по второму региону
model_1 = LinearRegression()
model_1.fit(features_train_1, target_train_1)
predict_1 = model_1.predict(features_valid_1)
rmse_1 = (mean_squared_error(target_valid_1, predict_1))**(0.5)
print('Средний запас предсказанного сырья по второму региону:', predict_1.mean())
print('Средний запас реального сырья по второму региону:', target_valid_1.mean())
print('RMSE модели:', rmse_1)

Средний запас предсказанного сырья по второму региону: 69.29357037294379
Средний запас реального сырья по второму региону: 69.29150044273759
RMSE модели: 0.8861463104896511


In [23]:
#обучим модель по третьему региону
model_2 = LinearRegression()
model_2.fit(features_train_2, target_train_2)
predict_2 = model_2.predict(features_valid_2)
rmse_2 = (mean_squared_error(target_valid_2, predict_2))**(0.5)
print('Средний запас предсказанного сырья по третьему региону:', predict_2.mean())
print('Средний запас реального сырья по третьему региону:', target_valid_2.mean())
print('RMSE модели:', rmse_2)

Средний запас предсказанного сырья по третьему региону: 95.14086176571024
Средний запас реального сырья по третьему региону: 94.68919659712621
RMSE модели: 40.185356062741405


### Вывод


Мы обучили модели для предсказания объема сырья по каждому региону. В третьем регионе среднее количество добываемого сырья больше, однако и ошибка его предсказания выше. Во втором регионе средний объем сырья меньше, но и точность предсказания гораздо меньше. Для выбора региона разработки пока недостаточно информации, нужно определить при какой добыче разработка не будет убыточной. Приступаем к следующему шагу: расчету прибыли от разработки.


[К оглавлению](#intro)

<a id='chapter3'></a>

# 3. Подготовка к расчёту прибыли


На этом этапе подготовим необходимую информацию для расчета прибыли от разработки месторождений: по условиям задачи нам известен общий бюджет на разработку и доход от продажи одного барреля сырья.


In [24]:
#объявление переменных
BUDGET = 10000000000
COST = 450
#расчет необходимого объема сырья
Voil = BUDGET/(COST*1000)
print('Объем сырья для безубыточной разработки:', Voil)

Объем сырья для безубыточной разработки: 22222.222222222223



Это общий объем сырья, который мы должны получить по региону. Известно, что в регионе мы можем разработать 200 скважин, значит средний объем сырья, полученный по каждой скважине должен быть:


In [25]:
print('Средний объем сырья для каждой скважины:', Voil/200)

Средний объем сырья для каждой скважины: 111.11111111111111


### Вывод


Видим, что среднее значение объема сырья, необходимое для безубыточной разработки месторождений, гораздо выше, чем среднее по каждому району. Теперь мы должны расчитать ту прибыль, которую получим от разработки лучших скважин по прогнозу, сделанному по построенной модели, а также возможные риски.


[К оглавлению](#intro)

<a id='chapter4'></a>

# 4. Расчёт прибыли и рисков 


Напишем функцию для расчета прибыли по выбранным скважинам.


In [26]:
#функция сортирует предсказания по модели и расчитывает прибыль по указанному количеству лучших скважин
def profit(target, prediction, prof, count, Voil):
    predictions = pd.Series(prediction)
    best_prediction = predictions.sort_values(ascending=False)
    target_best = target[best_prediction.index][:count]
    return target_best.sum()*prof-Voil*prof


Теперь посчитаем прибыль по трем регионам и посчитаем риски. Для этого применим технику Bootstrap, чтобы найти распределение прибыли в регионе. Затем проверим, является ли вероятность получения убытка меньше 2.5%. Если значение в квантиле 0.025 положительное, то количество отрицательных прибылей меньше, чем 2.5%.


In [27]:
#проведем бутстеп 1000 раз с выборкой в 500 скважин и посчитаем прибыль по 200 лучшим функцией profit
state = np.random.RandomState(0)
values = []
target_valid_0_new = target_valid_0.reset_index(drop=True)
predict_0_new = pd.Series(predict_0)
for i in range(1000):
    predict_subsample_0 = predict_0_new.sample(n=500, replace=True, random_state=state)
    target_subsample_0 = target_valid_0_new[predict_subsample_0.index]
    values.append(profit(target_subsample_0, predict_subsample_0, COST*1000, 200, Voil))
values = pd.Series(values)
#нижняя и верхняя граница доверительного интервала
lower = values.quantile(0.025)
upper = values.quantile(0.975)
#расчет рисков получить убытки
count = 0
for i in range(len(values)):
    if values[i]<0:
        count += 1
print('Среднее значение прибыли по первому региону', values.mean())
print('Значение нижней границы доверительного интервала:', lower)
print('Значение верхней границы доверительного интервала:', upper)
print('Риск получить убытки: {:.1%}'.format(count/len(values)))

Среднее значение прибыли по первому региону 472575925.6898031
Значение нижней границы доверительного интервала: -70667319.27551079
Значение верхней границы доверительного интервала: 1018205263.5521375
Риск получить убытки: 4.4%


In [28]:
state = np.random.RandomState(0)
values = []
target_valid_1_new = target_valid_1.reset_index(drop=True)
predict_1_new = pd.Series(predict_1)
for i in range(1000):
    predict_subsample_1 = predict_1_new.sample(n=500, replace=True, random_state=state)
    target_subsample_1 = target_valid_1_new[predict_subsample_1.index]
    values.append(profit(target_subsample_1, predict_subsample_1, COST*1000, 200, Voil))
values = pd.Series(values)
lower = values.quantile(0.025)
upper = values.quantile(0.975)
count = 0
for i in range(len(values)):
    if values[i]<0:
        count += 1
print('Среднее значение прибыли по второму региону', values.mean())
print('Значение нижней границы доверительного интервала:', lower)
print('Значение верхней границы доверительного интервала:', upper)
print('Риск получить убытки: {:.1%}'.format(count/len(values)))

Среднее значение прибыли по второму региону 555701118.6606231
Значение нижней границы доверительного интервала: 169469015.52352393
Значение верхней границы доверительного интервала: 991458470.5468459
Риск получить убытки: 0.3%


In [29]:
state = np.random.RandomState(0)
values = []
target_valid_2_new = target_valid_2.reset_index(drop=True)
predict_2_new = pd.Series(predict_2)
for i in range(1000):
    predict_subsample_2 = predict_2_new.sample(n=500, replace=True, random_state=state)
    target_subsample_2 = target_valid_2_new[predict_subsample_2.index]
    values.append(profit(target_subsample_2, predict_subsample_2, COST*1000, 200, Voil))
values = pd.Series(values)
lower = values.quantile(0.025)
upper = values.quantile(0.975)
count = 0
for i in range(len(values)):
    if values[i]<0:
        count += 1
print('Среднее значение прибыли по первому региону', values.mean())
print('Значение нижней границы доверительного интервала:', lower)
print('Значение верхней границы доверительного интервала:', upper)
print('Риск получить убытки: {:.1%}'.format(count/len(values)))

Среднее значение прибыли по первому региону 364725629.2418121
Значение нижней границы доверительного интервала: -160282369.874983
Значение верхней границы доверительного интервала: 933898025.2540721
Риск получить убытки: 9.7%


### Вывод


На данном этапе мы посчитали прибыль, которую получим, разрабатов 200 скважин по каждому региону, и посчитали риски. Только в одном регионе вероятность убытков меньше 2.5%, значит можно сделать вывод о необходимости разработки именно его.


[К оглавлению](#intro)

<a id='chapter5'></a>

# Общий вывод


В данной работе мы изучили работу скважин в трех разных регионах.

В качестве входных данных мы получили пробы нефти в скважинах региона, ее качество и объем запасов. Построив модель работы скважин, мы научились предсказывать полученный объем сырья в них. Таким образом, зная стоимость барреля нефти и бюджет на разработку месторождения, мы посчитали прибыль от каждого региона. Учитывая возможные риски при разработке, мы можем сделать вывод о предпочтительном регионе - это второй регион, где вероятность убытков меньше 2.5%.


[К оглавлению](#intro)